In [1]:
#### Dependencies
import os
import urllib
from urllib.request import urlopen
import json
import csv
import simplejson
import pandas as pd
import pytest
import base64
import sys
import glob
import time
import progressbar

bar = progressbar.ProgressBar()
for i in bar(range(100)):
    time.sleep(0.02)
    
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
from bs4 import BeautifulSoup
import requests
from pprint import pprint
from datetime import datetime
from ohmysportsfeedspy.v1_0 import API_v1_0
from ohmysportsfeedspy.v1_1 import API_v1_1
from ohmysportsfeedspy.v1_2 import API_v1_2
from ohmysportsfeedspy import MySportsFeeds
msf = MySportsFeeds(version="1.2")

100% (100 of 100) |#######################| Elapsed Time: 0:00:02 Time: 0:00:02


In [2]:
#Web Scrapping
quote_page = "https://www.pro-football-reference.com/super-bowl/"

#Query the website to get HTML page of the url
html_page = urlopen(quote_page)
html_page
#Parse the html using beautiful soup and store in a variable
sb_parser = BeautifulSoup(html_page, "html.parser")
superbowl_id = sb_parser.find(id = "all_super_bowls")
super_bowl_scores = superbowl_id.text.strip() # strip() is used to remove starting and trailing
# super_bowl_scores.to_json("Super_Bowl_Scores")
super_bowl_dumps = json.dumps(super_bowl_scores)
super_bowl_json = json.loads(super_bowl_dumps)
super_bowl_json.replace("\n", " ")
# super_bowl_json.to_json("Super_Bowl_Scores")
# print(super_bowl_json)
sb_DF = StringIO(super_bowl_json)
super_bowl_DF = sb_DF.writable()
#Using Pandas to webscrape
html_df, = pd.read_html(quote_page)
#Store as JSON and CSV
superbowl_DF = html_df.to_csv("Super_Bowl_Scores.csv", sep=",")

In [3]:
# Open CSV

sb_output = "Super_Bowl_Scores.csv"
# sb_open = open(sb_output, "r")
# sb_open_csv = sb_open.read()
# sb_open_csv
sb_open = pd.read_csv(sb_output)
new_col = ["Date", "Winner", "Pts", "Loser", "Pts.1", "MVP", "Stadium", "City", "State"]
del sb_open ["Unnamed: 0"]
sb_open.head()
# sb_open.set_index("SB")
# sb_csv_print = json.loads(sb_open_csv)
# sb_csv_print

,Date,SB,Winner,Pts,Loser,Pts.1,MVP,Stadium,City,State
0,"Feb 4, 2018",LII (52),Philadelphia Eagles,41,New England Patriots,33,Nick Foles,U.S. Bank Stadium,Minneapolis,Minnesota
1,"Feb 5, 2017",LI (51),New England Patriots,34,Atlanta Falcons,28,Tom Brady,NRG Stadium,Houston,Texas
2,"Feb 7, 2016",50,Denver Broncos,24,Carolina Panthers,10,Von Miller,Levi's Stadium,Santa Clara,California
3,"Feb 1, 2015",XLIX (49),New England Patriots,28,Seattle Seahawks,24,Tom Brady,University of Phoenix Stadium,Glendale,Arizona
4,"Feb 2, 2014",XLVIII (48),Seattle Seahawks,43,Denver Broncos,8,Malcolm Smith,MetLife Stadium,East Rutherford,New Jersey


In [4]:
#Open JSON
sb_output = "Super_Bowl_Scores.json"
sb_open = open(sb_output, "r")
sb_open_json = sb_open.read()
sb_json_print = json.loads(sb_open_json)
pprint(sb_json_print)
sb_date_pretty = sb_json_print["Date"]
# pprint(sb_date_pretty.keys())
#Came out much worse than pprint
# e = json.dumps(d, separators=(" ", ":"), sort_keys=True)
# e.replace(":", " ")
# # e.replace("\n", " ")

{'City': {'0': 'Minneapolis',
          '1': 'Houston',
          '10': 'Glendale',
          '11': 'Miami Gardens',
          '12': 'Detroit',
          '13': 'Jacksonville',
          '14': 'Houston',
          '15': 'San Diego',
          '16': 'New Orleans',
          '17': 'Tampa',
          '18': 'Atlanta',
          '19': 'Miami Gardens',
          '2': 'Santa Clara',
          '20': 'San Diego',
          '21': 'New Orleans',
          '22': 'Tempe',
          '23': 'Miami Gardens',
          '24': 'Atlanta',
          '25': 'Pasadena',
          '26': 'Minneapolis',
          '27': 'Tampa',
          '28': 'New Orleans',
          '29': 'Miami Gardens',
          '3': 'Glendale',
          '30': 'San Diego',
          '31': 'Pasadena',
          '32': 'New Orleans',
          '33': 'Palo Alto',
          '34': 'Tampa',
          '35': 'Pasadena',
          '36': 'Pontiac',
          '37': 'New Orleans',
          '38': 'Pasadena',
          '39': 'Miami',
          '4': 'East 

In [313]:
# SB_DF.from_csv("Super_Bowl_Scores.csv")

In [7]:
#SuperBowl DataFrame
sb_dates = []
sb_number = []
sb_mvp = []
sb_winner = []
sb_winPts = []
sb_loser = []
sb_losePts = []
sb_stadium = []
sb_city = []
sb_state = []

for data in sb_json_print:
    sb_dates.append(sb_json_print["Date"])
    sb_number.append(sb_json_print["SB"])
    sb_mvp.append(sb_json_print["MVP"])
    sb_winner.append(sb_json_print["Winner"])
    sb_winPts.append(sb_json_print["Pts"])
    sb_loser.append(sb_json_print["Loser"])
    sb_losePts.append(sb_json_print["Pts.1"])
    sb_stadium.append(sb_json_print["Stadium"])
    sb_city.append(sb_json_print["City"])
    sb_state.append(sb_json_print["State"])
    SB_DF = pd.DataFrame.from_dict(sb_json_print)
SB_DF = SB_DF[["SB", "Date", "Winner", "Pts", "Loser", "Pts.1", "MVP", "Stadium", "City", "State"]]
SB_dataDF = SB_DF.rename(index=str, columns={"Pts": "Winning Score", "Pts.1": "Losing Score"})
SB_dataDF
# sb_date_pretty = sb_json_print["Date"]
# pprint(sb_date_pretty)


,SB,Date,Winner,Winning Score,Loser,Losing Score,MVP,Stadium,City,State
0,LII (52),"Feb 4, 2018",Philadelphia Eagles,41,New England Patriots,33,Nick Foles,U.S. Bank Stadium,Minneapolis,Minnesota
1,LI (51),"Feb 5, 2017",New England Patriots,34,Atlanta Falcons,28,Tom Brady,NRG Stadium,Houston,Texas
10,XLII (42),"Feb 3, 2008",New York Giants,17,New England Patriots,14,Eli Manning,University of Phoenix Stadium,Glendale,Arizona
11,XLI (41),"Feb 4, 2007",Indianapolis Colts,29,Chicago Bears,17,Peyton Manning,Dolphin Stadium,Miami Gardens,Florida
12,XL (40),"Feb 5, 2006",Pittsburgh Steelers,21,Seattle Seahawks,10,Hines Ward,Ford Field,Detroit,Michigan
13,XXXIX (39),"Feb 6, 2005",New England Patriots,24,Philadelphia Eagles,21,Deion Branch,Alltel Stadium,Jacksonville,Florida
14,XXXVIII (38),"Feb 1, 2004",New England Patriots,32,Carolina Panthers,29,Tom Brady,Reliant Stadium,Houston,Texas
15,XXXVII (37),"Jan 26, 2003",Tampa Bay Buccaneers,48,Oakland Raiders,21,Dexter Jackson,Qualcomm Stadium,San Diego,California
16,XXXVI (36),"Feb 3, 2002",New England Patriots,20,St. Louis Rams,17,Tom Brady,Louisiana Superdome,New Orleans,Louisiana
17,XXXV (35),"Jan 28, 2001",Baltimore Ravens,34,New York Giants,7,Ray Lewis+,Raymond James Stadium,Tampa,Florida


In [8]:
NFL_data = "box_scores.csv"
NFL_data = pd.read_csv(NFL_data)
NFL_data.head(5)

,date,visitor,home,visitor_score,home_score,visitor_first_downs,visitor_rushing_first_downs,visitor_passing_first_downs,visitor_penalties,visitor_net_yards,...,home_kick_return_splits,home_int_return_splits,home_penalty_splits,home_fumble_splits,home_field_goals,home_third_down_splits,home_fourth_down_splits,home_total_plays,home_avg_gain,home_time_of_possession
0,"September 7, 2014",Cleveland,Pittsburgh,27,30,23,9,11,3,389,...,2-29,0-0,11-96,1-0,3-3,4-12-33%,1-1-100%,67,7.5,32:27
1,"September 7, 2014",Jacksonville,Philadelphia,17,34,18,2,14,2,306,...,1-24,0-0,6-50,3-2,2-2,8-19-42%,1-1-100%,82,5.1,30:46
2,"September 4, 2014",Green Bay,Seattle,16,36,19,4,13,2,255,...,3-60,1-21,4-69,2-1,2-2,4-11-36%,1-1-100%,66,6.0,33:20
3,"September 7, 2014",Minnesota,St. Louis,34,6,18,6,10,2,355,...,1-26,0-0,13-121,4-0,2-3,4-14-28%,0-0-0%,63,5.0,31:43
4,"September 7, 2014",Cincinnati,Baltimore,23,16,16,4,11,1,380,...,4-109,0-0,3-29,2-1,1-2,8-17-47%,1-2-50%,85,5.0,29:30


In [9]:
team_scores = NFL_data[["date","visitor","home","visitor_score","home_score"]]
team_scores.head(5)

,date,visitor,home,visitor_score,home_score
0,"September 7, 2014",Cleveland,Pittsburgh,27,30
1,"September 7, 2014",Jacksonville,Philadelphia,17,34
2,"September 4, 2014",Green Bay,Seattle,16,36
3,"September 7, 2014",Minnesota,St. Louis,34,6
4,"September 7, 2014",Cincinnati,Baltimore,23,16


In [10]:
team_scores.nlargest(5, "home_score")

,date,visitor,home,visitor_score,home_score
936,"October 23, 2011",Indianapolis,New Orleans,7,62
1190,"October 18, 2009",Tennessee,New England,0,59
1773,"November 18, 2012",Indianapolis,New England,24,59
2558,"December 9, 2012",Arizona,Seattle,0,58
3395,"November 28, 2004",Cleveland,Cincinnati,48,58


In [11]:
team_scores.nlargest(5, "visitor_score")

,date,visitor,home,visitor_score,home_score
744,"October 24, 2010",Oakland,Denver,59,14
1049,"November 15, 2010",Philadelphia,Washington,59,28
3103,"December 15, 2013",Kansas City,Oakland,56,31
3261,"November 18, 2007",New England,Buffalo,56,10
1854,"September 28, 2003",Indianapolis,New Orleans,55,21


In [12]:
NFL_games = "nfl_games.csv"
NFL_games = pd.read_csv(NFL_games)
NFL_date_games = NFL_games.set_index("date")

NFL_date_games = NFL_date_games["2014-09-04":]

In [13]:
NFL_date_games = NFL_date_games[["season","team1","team2","score1","score2"]]
nfl_scores = NFL_date_games
nfl_scores.head()

,season,team1,team2,score1,score2
date,,,,,
2014-09-04,2014,SEA,GB,36,16
2014-09-07,2014,PHI,JAX,34,17
2014-09-07,2014,PIT,CLE,30,27
2014-09-07,2014,BAL,CIN,16,23
2014-09-07,2014,TB,CAR,14,20


In [14]:
team1_scores = nfl_scores.groupby("team1").sum()["score1"]
team2_scores = nfl_scores.groupby("team2").sum()["score2"]

team1_scores_df = pd.DataFrame(team1_scores)
team2_scores_df = pd.DataFrame(team2_scores)

In [15]:
#Scores from 2014 to 2017
nfl_team_scores = pd.concat([team1_scores_df, team2_scores_df], axis=1)
nfl_team_scores["Total Points from 2014 to 2017"] = nfl_team_scores["score1"] + nfl_team_scores["score2"]
del nfl_team_scores["score1"]
del nfl_team_scores["score2"]
nfl_team_scores.nlargest(10, "Total Points from 2014 to 2017")

,Total Points from 2014 to 2017
NE,1631
GB,1482
PIT,1374
ATL,1368
CAR,1342
SEA,1334
NO,1278
ARI,1274
IND,1259
DEN,1250


In [16]:
# Obtained all scores for season, including full season, called DataFrames on all.

nfl_season_2014 = NFL_date_games["2014-09-04":"2015-02-01"]
scores_2014_team1 = nfl_season_2014.groupby("team1").sum()["score1"]
scores_2014_team2 = nfl_season_2014.groupby("team2").sum()["score2"]
scores_2014_df1 = pd.DataFrame(scores_2014_team1)
scores_2014_df2 = pd.DataFrame(scores_2014_team2)
scores_2014 = pd.concat([scores_2014_df1, scores_2014_df2], axis = 1)
scores_2014["Total Points for 2014"] = scores_2014["score1"] + scores_2014["score2"]
del scores_2014["score1"]
del scores_2014["score2"]

nfl_season_2015 = NFL_date_games["2015-09-10":"2016-02-07"]
scores_2015_team1 = nfl_season_2015.groupby("team1").sum()["score1"]
scores_2015_team2 = nfl_season_2015.groupby("team2").sum()["score2"]
scores_2015_df1 = pd.DataFrame(scores_2015_team1)
scores_2015_df2 = pd.DataFrame(scores_2015_team2)
scores_2015 = pd.concat([scores_2015_df1, scores_2015_df2], axis = 1)
scores_2015["Total Points for 2015"] = scores_2015["score1"] + scores_2015["score2"]
del scores_2015["score1"]
del scores_2015["score2"]


nfl_season_2016 = NFL_date_games["2016-09-08":"2017-02-05"]
scores_2016_team1 = nfl_season_2016.groupby("team1").sum()["score1"]
scores_2016_team2 = nfl_season_2016.groupby("team2").sum()["score2"]
scores_2016_df1 = pd.DataFrame(scores_2016_team1)
scores_2016_df2 = pd.DataFrame(scores_2016_team2)
scores_2016 = pd.concat([scores_2016_df1, scores_2016_df2], axis = 1)
scores_2016["Total Points for 2016"] = scores_2016["score1"] + scores_2016["score2"]
del scores_2016["score1"]
del scores_2016["score2"]


nfl_team_scores
scores_2014
scores_2015
scores_2016

,Total Points for 2016
ARI,418
ATL,648
BAL,343
BUF,399
CAR,369
CHI,279
CIN,325
CLE,264
DAL,452
DEN,333


In [17]:
#Getting final row for sums for each DF

nfl_team_scores_sum = nfl_team_scores.loc['Total', 'Sum of Point']= nfl_team_scores['Total Points from 2014 to 2017'].sum()
scores_2014_sum = scores_2014.loc['Total', 'Sum of Point']= scores_2014['Total Points for 2014'].sum()
scores_2015_sum = scores_2015.loc['Total', 'Sum of Point']= scores_2015['Total Points for 2015'].sum()
scores_2016_sum = scores_2016.loc['Total', 'Sum of Point']= scores_2016['Total Points for 2016'].sum()

In [18]:
sum_all = nfl_team_scores["Total Points from 2014 to 2017"].sum()
sum_all

36438.0

# Cumulative Player Stats
### Will Be Merged to Analyze Team Stats

In [59]:
nfl_stats = "Cumulative_Stats_Combined.csv"
nfl_Stats = pd.read_csv(nfl_stats)
nfl_Stats.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1, inplace=True)
nfl_Stats.set_index("Season")
# nfl_Stats.to_csv("NFL_Cumulative_Stats_Combined.csv")

,#Player ID,#LastName,#FirstName,#Jersey Num,#Position,#Height,#Weight,#Birth Date,#Age,#Birth City,...,#PuntDown,#PuntIn20,#PuntIn20Pct,#PuntTB,#PuntTBPct,#PuntFC,#PuntRet,#PuntRetYds,#PuntRetAvg,#GamesStarted
Season,,,,,,,,,,,,,,,,,,,,,
2014-2015 Regular,7350,Abdullah,Husain,39.0,DB,"6'0""",204.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2014-2015 Regular,6899,Abdul-Quddus,Isa,42.0,DB,"6'1""",201.0,1989-08-03,28.0,"Newark, NJ",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2014-2015 Regular,7861,Aboushi,Oday,75.0,G,"6'5""",315.0,1991-06-05,26.0,"Brooklyn, NY",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2014-2015 Regular,8062,Acho,Emmanuel,51.0,LB,"6'2""",240.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2014-2015 Regular,6430,Acho,Sam,49.0,OLB,"6'3""",257.0,1988-09-06,29.0,"Dallas, TX",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2014-2015 Regular,8459,Acosta,Corey,25.0,K,"6'0""",190.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-2015 Regular,6924,Adams,Davante,17.0,WR,"6'1""",215.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2014-2015 Regular,7039,Adams,Jeff,70.0,OT,"6'7""",305.0,1989-09-06,28.0,"Berwyn, IL",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2014-2015 Regular,8134,Adams,Mike,76.0,OT,"6'7""",323.0,1990-03-10,28.0,"Farrell, PA",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


#  Building Team Stats

In [130]:
#Sort Stat Leaders by Team
nfl_Stats.groupby(nfl_Stats.index).sum()
teamTotals = nfl_Stats.reset_index().groupby("#Team Abbr.").sum()
teamTotals.keys()

Index(['index', '#Player ID', '#Jersey Num', '#Weight', '#Age', '#Team ID',
       '#GamesPlayed', '#PassAttempts', '#PassCompletions', '#PassPct',
       ...
       '#PuntDown', '#PuntIn20', '#PuntIn20Pct', '#PuntTB', '#PuntTBPct',
       '#PuntFC', '#PuntRet', '#PuntRetYds', '#PuntRetAvg', '#GamesStarted'],
      dtype='object', length=145)

##  Offensive Categories

In [133]:
#Offensive Totals
offTotals = teamTotals[["#PassAttempts", "#PassCompletions", "#PassPct", "#PassAttemptsPerGame", 
                        "#PassYards", "#PassYardsPerAtt", "#PassYardsPerGame", "#PassTD", "#PassTDPct", 
                        "#PassInt", "#PassIntPct", "#PassLng", "#PassAvg", "#Pass20Plus", 
                        "#Pass40Plus", "#PassSacks", "#PassSackY", "#QBRating", "#RushAttempts", 
                        "#RushAttemptsPerGame", "#RushYards", "#RushAverage", "#RushYardsPerGame", "#RushTD", 
                        "#RushLng", "#Rush1stDowns", "#Rush1stDownsPct", "#Rush20Plus", "#Rush40Plus", 
                        "#RushFumbles", "#Targets", "#Receptions", "#RecYards", "#RecAverage", 
                        "#RecYardsPerGame", "#RecTD", "#RecLng", "#Rec1stDowns", "#Rec20Plus", 
                        "#Rec40Plus", "#RecFumbles"
                       ]]
offTotals.head()

,#PassAttempts,#PassCompletions,#PassPct,#PassAttemptsPerGame,#PassYards,#PassYardsPerAtt,#PassYardsPerGame,#PassTD,#PassTDPct,#PassInt,...,#Receptions,#RecYards,#RecAverage,#RecYardsPerGame,#RecTD,#RecLng,#Rec1stDowns,#Rec20Plus,#Rec40Plus,#RecFumbles
#Team Abbr.,,,,,,,,,,,,,,,,,,,,,
ARI,2468.0,1445.0,860.1,322.8,17712.0,113.5,2119.6,110.0,56.5,65.0,...,1375.0,16983.0,799.0,1608.7,106.0,3918.0,843.0,208.0,38.0,0.0
ATL,2486.0,1658.0,654.4,218.3,19903.0,122.7,1773.6,119.0,131.0,47.0,...,1625.0,19650.0,776.2,1910.3,117.0,3684.0,970.0,240.0,46.0,0.0
BAL,2739.0,1713.0,600.2,298.7,17463.0,59.4,1902.3,99.0,42.8,69.0,...,1447.0,14301.0,604.3,1285.3,73.0,1850.0,699.0,141.0,24.0,0.0
BUF,2053.0,1251.0,721.0,204.0,13982.0,77.5,1292.8,79.0,27.7,39.0,...,1265.0,14317.0,815.4,1284.1,82.0,3485.0,676.0,170.0,33.0,0.0
CAR,2307.0,1344.0,636.5,263.1,16538.0,76.0,1950.0,110.0,42.6,60.0,...,1283.0,15801.0,868.4,1849.3,107.0,3826.0,791.0,192.0,32.0,0.0


In [136]:
#Passing Offense
passOff = offTotals[["#PassAttempts", "#PassCompletions", "#PassPct", "#PassAttemptsPerGame", 
                     "#PassYards", "#PassYardsPerAtt", "#PassYardsPerGame", "#PassTD", "#PassTDPct", 
                     "#PassInt", "#PassIntPct", "#PassLng", "#PassAvg", "#Pass20Plus", 
                     "#Pass40Plus", "#PassSacks", "#PassSackY", "#QBRating"]]
passOff.head()

,#PassAttempts,#PassCompletions,#PassPct,#PassAttemptsPerGame,#PassYards,#PassYardsPerAtt,#PassYardsPerGame,#PassTD,#PassTDPct,#PassInt,#PassIntPct,#PassLng,#PassAvg,#Pass20Plus,#Pass40Plus,#PassSacks,#PassSackY,#QBRating
#Team Abbr.,,,,,,,,,,,,,,,,,,
ARI,2468.0,1445.0,860.1,322.8,17712.0,113.5,2119.6,110.0,56.5,65.0,48.6,1263.0,113.5,234.0,38.0,157.0,1119.0,4752.7
ATL,2486.0,1658.0,654.4,218.3,19903.0,122.7,1773.6,119.0,131.0,47.0,47.1,1107.0,122.7,252.0,47.0,137.0,976.0,4445.1
BAL,2739.0,1713.0,600.2,298.7,17463.0,59.4,1902.3,99.0,42.8,69.0,39.8,548.0,59.4,189.0,33.0,112.0,849.0,3980.7
BUF,2053.0,1251.0,721.0,204.0,13982.0,77.5,1292.8,79.0,27.7,39.0,80.6,1108.0,77.5,179.0,32.0,176.0,1040.0,4059.4
CAR,2307.0,1344.0,636.5,263.1,16538.0,76.0,1950.0,110.0,42.6,60.0,25.1,1192.0,76.0,211.0,33.0,163.0,1236.0,4795.3


In [138]:
#Rushing Offense
rushOff = offTotals[["#RushAttemptsPerGame", "#RushYards", "#RushAverage", "#RushYardsPerGame", "#RushTD", 
                     "#RushLng", "#Rush1stDowns", "#Rush1stDownsPct", "#Rush20Plus", "#Rush40Plus", 
                     "#RushFumbles"]]
rushOff.head()

,#RushAttemptsPerGame,#RushYards,#RushAverage,#RushYardsPerGame,#RushTD,#RushLng,#Rush1stDowns,#Rush1stDownsPct,#Rush20Plus,#Rush40Plus,#RushFumbles
#Team Abbr.,,,,,,,,,,,
ARI,179.8,6188.0,207.0,623.4,49.0,1114.0,323.0,1279.2,30.0,9.0,33.0
ATL,170.8,7240.0,174.3,693.6,60.0,1332.0,403.0,1053.7,40.0,9.0,31.0
BAL,165.6,7064.0,157.3,631.0,50.0,824.0,407.0,1102.8,47.0,7.0,27.0
BUF,179.8,8598.0,177.1,810.1,66.0,1363.0,445.0,1492.9,65.0,13.0,29.0
CAR,252.5,8831.0,263.5,1032.6,67.0,1767.0,516.0,1691.3,37.0,11.0,33.0


In [139]:
#Receiving Offense
recOff = offTotals[["#Targets", "#Receptions", "#RecYards", "#RecAverage", 
                    "#RecYardsPerGame", "#RecTD", "#RecLng", "#Rec1stDowns", "#Rec20Plus", 
                    "#Rec40Plus", "#RecFumbles"]]
recOff.head()

,#Targets,#Receptions,#RecYards,#RecAverage,#RecYardsPerGame,#RecTD,#RecLng,#Rec1stDowns,#Rec20Plus,#Rec40Plus,#RecFumbles
#Team Abbr.,,,,,,,,,,,
ARI,2290.0,1375.0,16983.0,799.0,1608.7,106.0,3918.0,843.0,208.0,38.0,0.0
ATL,2405.0,1625.0,19650.0,776.2,1910.3,117.0,3684.0,970.0,240.0,46.0,0.0
BAL,2238.0,1447.0,14301.0,604.3,1285.3,73.0,1850.0,699.0,141.0,24.0,0.0
BUF,2052.0,1265.0,14317.0,815.4,1284.1,82.0,3485.0,676.0,170.0,33.0,0.0
CAR,2173.0,1283.0,15801.0,868.4,1849.3,107.0,3826.0,791.0,192.0,32.0,0.0


In [170]:
offScores = offTotals[["#PassTD", "#RushTD", "#RecTD"]]
offScores

,#PassTD,#RushTD,#RecTD
#Team Abbr.,,,
ARI,110.0,49.0,106.0
ATL,119.0,60.0,117.0
BAL,99.0,50.0,73.0
BUF,79.0,66.0,82.0
CAR,110.0,67.0,107.0
CHI,82.0,44.0,82.0
CIN,95.0,61.0,95.0
CLE,62.0,42.0,60.0
DAL,105.0,67.0,107.0


In [171]:
offTurn_Sack = offTotals[["#PassInt", "#PassIntPct", "#PassSacks", "#PassSackY", "#RushFumbles", "#RecFumbles"]]
offTurn_Sack.head()

,#PassInt,#PassIntPct,#PassSacks,#PassSackY,#RushFumbles,#RecFumbles
#Team Abbr.,,,,,,
ARI,65.0,48.6,157.0,1119.0,33.0,0.0
ATL,47.0,47.1,137.0,976.0,31.0,0.0
BAL,69.0,39.8,112.0,849.0,27.0,0.0
BUF,39.0,80.6,176.0,1040.0,29.0,0.0
CAR,60.0,25.1,163.0,1236.0,33.0,0.0


## Defensive Categories

In [148]:
defTotals = teamTotals[["#TackleSolo", "#TackleTotal", "#TackleAst", "#Sacks", "#SackYds", 
                        "#TacklesForLoss", "#Interceptions", "#IntTD", "#IntYds", "#IntAverage", 
                        "#IntLng", "#PassesDefended", "#Stuffs", "#Safeties", "#StuffYds",
                        "#Fumbles", "#FumLost", "#FumForced", "#FumOwnRec", "#FumOppRec", 
                        "#FumRecYds", "#FumTotalRec", "#FumTD"
]]
defTotals.head()

,#TackleSolo,#TackleTotal,#TackleAst,#Sacks,#SackYds,#TacklesForLoss,#Interceptions,#IntTD,#IntYds,#IntAverage,...,#StuffYds,#KB,#Fumbles,#FumLost,#FumForced,#FumOwnRec,#FumOppRec,#FumRecYds,#FumTotalRec,#FumTD
#Team Abbr.,,,,,,,,,,,,,,,,,,,,,
ARI,3171.0,3956.0,785.0,156.0,1232.0,527.0,68.0,10.0,756.0,487.4,...,1863.0,5.0,91.0,40.0,66.0,41.0,45.0,-205.0,86.0,5.0
ATL,2995.0,4243.0,1248.0,130.0,926.0,394.0,54.0,6.0,631.0,388.3,...,1343.0,5.0,75.0,31.0,56.0,40.0,40.0,-5.0,80.0,9.0
BAL,2811.0,4054.0,1243.0,168.0,1279.0,463.0,59.0,5.0,561.0,265.4,...,1696.0,10.0,66.0,26.0,51.0,35.0,43.0,-230.0,78.0,8.0
BUF,2981.0,4229.0,1248.0,139.0,904.0,452.0,63.0,4.0,939.0,430.5,...,1393.0,1.0,74.0,30.0,50.0,40.0,32.0,-81.0,72.0,7.0
CAR,2821.0,4235.0,1414.0,200.5,1469.0,529.0,74.0,9.0,602.0,340.7,...,2020.0,4.0,79.0,42.0,77.0,33.0,54.0,-276.0,87.0,4.0


In [150]:
passDef = defTotals[["#TackleSolo", "#TackleTotal", "#TackleAst", "#Sacks", "#SackYds",
                     "#Interceptions", "#IntTD", "#IntYds", "#IntAverage", "#IntLng", 
                     "#PassesDefended", "#Fumbles", "#FumLost", "#FumForced", "#FumOwnRec", 
                     "#FumOppRec", "#FumRecYds", "#FumTotalRec", "#FumTD", "#Safeties"]]
passDef.head()

,#TackleSolo,#TackleTotal,#TackleAst,#Sacks,#SackYds,#Interceptions,#IntTD,#IntYds,#IntAverage,#IntLng,#PassesDefended,#Fumbles,#FumLost,#FumForced,#FumOwnRec,#FumOppRec,#FumRecYds,#FumTotalRec,#FumTD,#Safeties
#Team Abbr.,,,,,,,,,,,,,,,,,,,,
ARI,3171.0,3956.0,785.0,156.0,1232.0,68.0,10.0,756.0,487.4,657.0,278.0,91.0,40.0,66.0,41.0,45.0,-205.0,86.0,5.0,4.0
ATL,2995.0,4243.0,1248.0,130.0,926.0,54.0,6.0,631.0,388.3,622.0,301.0,75.0,31.0,56.0,40.0,40.0,-5.0,80.0,9.0,1.0
BAL,2811.0,4054.0,1243.0,168.0,1279.0,59.0,5.0,561.0,265.4,400.0,300.0,66.0,26.0,51.0,35.0,43.0,-230.0,78.0,8.0,2.0
BUF,2981.0,4229.0,1248.0,139.0,904.0,63.0,4.0,939.0,430.5,766.0,339.0,74.0,30.0,50.0,40.0,32.0,-81.0,72.0,7.0,0.0
CAR,2821.0,4235.0,1414.0,200.5,1469.0,74.0,9.0,602.0,340.7,575.0,291.0,79.0,42.0,77.0,33.0,54.0,-276.0,87.0,4.0,1.0


In [151]:
rushDef = defTotals[["#TackleSolo", "#TackleTotal", "#TackleAst", "#Sacks", "#SackYds", 
                     "#TacklesForLoss", "#Stuffs", "#Safeties", "#StuffYds", "#Fumbles", 
                     "#FumLost", "#FumForced", "#FumOwnRec", "#FumOppRec", 
                     "#FumRecYds", "#FumTotalRec", "#FumTD"]]
rushDef.head()

,#TackleSolo,#TackleTotal,#TackleAst,#Sacks,#SackYds,#TacklesForLoss,#Stuffs,#Safeties,#StuffYds,#Fumbles,#FumLost,#FumForced,#FumOwnRec,#FumOppRec,#FumRecYds,#FumTotalRec,#FumTD
#Team Abbr.,,,,,,,,,,,,,,,,,
ARI,3171.0,3956.0,785.0,156.0,1232.0,527.0,620.0,4.0,1863.0,91.0,40.0,66.0,41.0,45.0,-205.0,86.0,5.0
ATL,2995.0,4243.0,1248.0,130.0,926.0,394.0,515.0,1.0,1343.0,75.0,31.0,56.0,40.0,40.0,-5.0,80.0,9.0
BAL,2811.0,4054.0,1243.0,168.0,1279.0,463.0,550.0,2.0,1696.0,66.0,26.0,51.0,35.0,43.0,-230.0,78.0,8.0
BUF,2981.0,4229.0,1248.0,139.0,904.0,452.0,636.0,0.0,1393.0,74.0,30.0,50.0,40.0,32.0,-81.0,72.0,7.0
CAR,2821.0,4235.0,1414.0,200.5,1469.0,529.0,670.0,1.0,2020.0,79.0,42.0,77.0,33.0,54.0,-276.0,87.0,4.0


In [152]:
turnDef = defTotals[["#Interceptions", "#IntTD", "#IntYds", "#IntAverage", "#IntLng",
                     "#Fumbles", "#FumLost", "#FumForced", "#FumOwnRec", "#FumOppRec", 
                     "#FumRecYds", "#FumTotalRec", "#FumTD"]]
turnDef.head()

,#Interceptions,#IntTD,#IntYds,#IntAverage,#IntLng,#Fumbles,#FumLost,#FumForced,#FumOwnRec,#FumOppRec,#FumRecYds,#FumTotalRec,#FumTD
#Team Abbr.,,,,,,,,,,,,,
ARI,68.0,10.0,756.0,487.4,657.0,91.0,40.0,66.0,41.0,45.0,-205.0,86.0,5.0
ATL,54.0,6.0,631.0,388.3,622.0,75.0,31.0,56.0,40.0,40.0,-5.0,80.0,9.0
BAL,59.0,5.0,561.0,265.4,400.0,66.0,26.0,51.0,35.0,43.0,-230.0,78.0,8.0
BUF,63.0,4.0,939.0,430.5,766.0,74.0,30.0,50.0,40.0,32.0,-81.0,72.0,7.0
CAR,74.0,9.0,602.0,340.7,575.0,79.0,42.0,77.0,33.0,54.0,-276.0,87.0,4.0


In [169]:
defScores = defTotals[["#IntTD", "#FumTD"]]
defScores

,#IntTD,#FumTD
#Team Abbr.,,
ARI,10.0,5.0
ATL,6.0,9.0
BAL,5.0,8.0
BUF,4.0,7.0
CAR,9.0,4.0
CHI,4.0,2.0
CIN,5.0,1.0
CLE,6.0,1.0
DAL,4.0,2.0


## Special Teams Categories

In [145]:
sptTotals = teamTotals[["#KB", "#KrRet", "#KrYds", "#KrAvg", "#KrLng","#KrTD", 
                        "#Kr20Plus", "#Kr40Plus", "#KrFC", "#KrFum", "#PrRet", 
                        "#PrYds", "#PrAvg", "#PrLng", "#PrTD", "#Pr20Plus", 
                        "#Pr40Plus", "#PrFC", "#PrFum", "#FgBlk", "#FgMade", 
                        "#FgAtt","#FgPct", "#FgMade1_19", "#FgAtt1_19", "#Fg1_19Pct",
                        "#FgMade20_29", "#FgAtt20_29", "#Fg20_29Pct", "#FgMade30_39", "#FgAtt30_39",
                        "#Fg30_39Pct","#FgMade40_49", "#FgAtt40_49", "#Fg40_49Pct", "#FgMade50Plus", "#FgAtt50Plus",
                        "#Fg50PlusPct", "#FgLng", "#XpBlk", "#XpMade", "#XpAtt", 
                        "#XpPct", "#FgAndXpPts", "#KoPct", "#Kickoffs", "#KoYds",
                        "#KoOOB", '#KoAvg', "#KoTB", "#KoRet", "#KoRetYds", 
                        "#KoRetAvgYds", "#KoTD", "#KoOS", "#KoOSR", "#Punts", 
                        "#PuntYds", "#PuntNetYds", "#PuntLng", "#PuntAvg", "#PuntNetAvg", 
                        "#PuntBlk", "#PuntOOB", "#PuntDown", "#PuntIn20", "#PuntIn20Pct", 
                        "#PuntTB", "#PuntTBPct", "#PuntFC", "#PuntRet", "#PuntRetYds", "#PuntRetAvg"
]]
sptTotals.head()

,#KrRet,#KrYds,#KrAvg,#KrLng,#KrTD,#Kr20Plus,#Kr40Plus,#KrFC,#KrFum,#PrRet,...,#PuntOOB,#PuntDown,#PuntIn20,#PuntIn20Pct,#PuntTB,#PuntTBPct,#PuntFC,#PuntRet,#PuntRetYds,#PuntRetAvg
#Team Abbr.,,,,,,,,,,,,,,,,,,,,,
ARI,122.0,2732.0,385.1,856.0,1.0,69.0,7.0,0.0,0.0,142.0,...,0.0,39.0,107.0,258.4,22.0,44.8,106.0,139.0,1687.0,94.4
ATL,146.0,3414.0,277.4,898.0,0.0,90.0,9.0,0.0,0.0,97.0,...,0.0,16.0,91.0,241.7,12.0,29.6,94.0,106.0,1166.0,70.4
BAL,129.0,3386.0,398.4,720.0,2.0,93.0,11.0,0.0,0.0,121.0,...,0.0,38.0,101.0,170.0,17.0,29.2,58.0,118.0,1064.0,50.5
BUF,153.0,3272.0,472.4,984.0,1.0,85.0,4.0,0.0,0.0,116.0,...,0.0,38.0,82.0,164.3,19.0,42.8,67.0,146.0,1121.0,30.4
CAR,130.0,2971.0,519.8,1119.0,1.0,77.0,7.0,0.0,0.0,129.0,...,0.0,43.0,103.0,286.7,24.0,108.2,87.0,136.0,1613.0,116.5


In [166]:
puntRETtotals = teamTotals[["#PrRet", 
                        "#PrYds", "#PrAvg", "#PrLng", "#PrTD", "#Pr20Plus", 
                        "#Pr40Plus", "#PrFC", "#PrFum","#PuntRet", "#PuntRetYds", "#PuntRetAvg"]]
puntRETtotals

,#PrRet,#PrYds,#PrAvg,#PrLng,#PrTD,#Pr20Plus,#Pr40Plus,#PrFC,#PrFum,#PuntRet,#PuntRetYds,#PuntRetAvg
#Team Abbr.,,,,,,,,,,,,
ARI,142.0,1166.0,89.2,449.0,1.0,7.0,2.0,114.0,0.0,139.0,1687.0,94.4
ATL,97.0,1171.0,68.9,418.0,1.0,13.0,6.0,81.0,0.0,106.0,1166.0,70.4
BAL,121.0,1375.0,145.8,356.0,2.0,14.0,4.0,84.0,0.0,118.0,1064.0,50.5
BUF,116.0,1318.0,130.4,542.0,1.0,20.0,5.0,79.0,0.0,146.0,1121.0,30.4
CAR,129.0,1287.0,86.3,442.0,2.0,12.0,3.0,108.0,0.0,136.0,1613.0,116.5
CHI,106.0,1141.0,215.2,452.0,3.0,14.0,5.0,78.0,0.0,124.0,1586.0,62.5
CIN,162.0,1735.0,162.2,657.0,0.0,23.0,7.0,68.0,0.0,146.0,1311.0,48.9
CLE,114.0,1046.0,93.2,286.0,1.0,9.0,2.0,67.0,0.0,162.0,1828.0,45.0
DAL,119.0,1048.0,82.5,410.0,1.0,11.0,1.0,69.0,0.0,53.0,438.0,26.5


In [165]:
puntTotals = teamTotals[["#PuntYds", "#PuntNetYds", "#PuntLng", "#PuntAvg", "#PuntNetAvg", 
                        "#PuntBlk", "#PuntOOB", "#PuntDown", "#PuntIn20", "#PuntIn20Pct", 
                        "#PuntTB", "#PuntTBPct", "#PuntFC"]]
puntTotals

,#PuntYds,#PuntNetYds,#PuntLng,#PuntAvg,#PuntNetAvg,#PuntBlk,#PuntOOB,#PuntDown,#PuntIn20,#PuntIn20Pct,#PuntTB,#PuntTBPct,#PuntFC
#Team Abbr.,,,,,,,,,,,,,
ARI,13644.0,11957.0,1331.0,439.9,394.4,2.0,0.0,39.0,107.0,258.4,22.0,44.8,106.0
ATL,10591.0,9425.0,1231.0,305.1,272.5,1.0,0.0,16.0,91.0,241.7,12.0,29.6,94.0
BAL,11396.0,10332.0,336.0,240.2,208.3,2.0,0.0,38.0,101.0,170.0,17.0,29.2,58.0
BUF,12649.0,11528.0,1131.0,225.6,209.1,0.0,0.0,38.0,82.0,164.3,19.0,42.8,67.0
CAR,13417.0,11804.0,1483.0,402.1,356.8,2.0,0.0,43.0,103.0,286.7,24.0,108.2,87.0
CHI,12313.0,10727.0,1020.0,226.4,192.7,1.0,0.0,32.0,89.0,166.3,18.0,26.5,87.0
CIN,13859.0,12548.0,1174.0,278.3,252.8,2.0,0.0,46.0,106.0,208.7,23.0,33.2,77.0
CLE,14148.0,12320.0,262.0,186.1,161.6,1.0,0.0,42.0,93.0,123.8,17.0,21.8,77.0
DAL,7403.0,6965.0,476.0,284.1,275.0,1.0,0.0,26.0,75.0,307.9,13.0,76.0,64.0


In [164]:
krTotals = teamTotals[["#KrRet", "#KrYds", "#KrAvg", "#KrLng","#KrTD", 
                        "#Kr20Plus", "#Kr40Plus", "#KrFC", "#KrFum","#KoRet", "#KoRetYds", 
                        "#KoRetAvgYds", "#KoTD", "#KoOS", "#KoOSR"]]
krTotals.head()

,#KrRet,#KrYds,#KrAvg,#KrLng,#KrTD,#Kr20Plus,#Kr40Plus,#KrFC,#KrFum,#KoRet,#KoRetYds,#KoRetAvgYds,#KoTD,#KoOS,#KoOSR
#Team Abbr.,,,,,,,,,,,,,,,
ARI,122.0,2732.0,385.1,856.0,1.0,69.0,7.0,0.0,0.0,150.0,3870.0,126.5,0.0,0.0,0.0
ATL,146.0,3414.0,277.4,898.0,0.0,90.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BAL,129.0,3386.0,398.4,720.0,2.0,93.0,11.0,0.0,0.0,100.0,2191.0,114.2,0.0,0.0,0.0
BUF,153.0,3272.0,472.4,984.0,1.0,85.0,4.0,0.0,0.0,136.0,3017.0,154.5,0.0,0.0,0.0
CAR,130.0,2971.0,519.8,1119.0,1.0,77.0,7.0,0.0,0.0,93.0,2374.0,157.3,0.0,0.0,0.0


In [161]:
kickOFFtotals = teamTotals[["#KoPct", "#Kickoffs", "#KoYds",
                        "#KoOOB", '#KoAvg', "#KoTB"]]
kickOFFtotals.head()

,#KoPct,#Kickoffs,#KoYds,#KoOOB,#KoAvg,#KoTB
#Team Abbr.,,,,,,
ARI,0.0,336.0,21937.0,0.0,456.6,186.0
ATL,0.0,0.0,0.0,0.0,0.0,0.0
BAL,0.0,335.0,21714.0,0.0,325.1,235.0
BUF,0.0,301.0,19255.0,0.0,507.5,165.0
CAR,0.0,378.0,24646.0,0.0,458.6,285.0


In [153]:
blockTotals = teamTotals[["#KB", "#FgBlk", "#XpBlk", "#PuntBlk"]]
blockTotals.head()

,#KB,#FgBlk,#XpBlk,#PuntBlk
#Team Abbr.,,,,
ARI,5.0,3.0,3.0,2.0
ATL,5.0,5.0,0.0,1.0
BAL,10.0,3.0,0.0,2.0
BUF,1.0,1.0,1.0,0.0
CAR,4.0,5.0,4.0,2.0


In [156]:
fg_xpTotals = teamTotals[["#FgMade", "#FgAtt","#FgPct", "#FgMade1_19", "#FgAtt1_19", "#Fg1_19Pct",
                          "#FgMade20_29", "#FgAtt20_29", "#Fg20_29Pct", "#FgMade30_39", "#FgAtt30_39",
                          "#Fg30_39Pct","#FgMade40_49", "#FgAtt40_49", "#Fg40_49Pct", "#FgMade50Plus", 
                          "#FgAtt50Plus", "#Fg50PlusPct", "#FgLng", "#XpMade", "#XpAtt", 
                          "#XpPct", "#FgAndXpPts",]]
fg_xpTotals.head()

,#FgMade,#FgAtt,#FgPct,#FgMade1_19,#FgAtt1_19,#Fg1_19Pct,#FgMade20_29,#FgAtt20_29,#Fg20_29Pct,#FgMade30_39,...,#FgAtt40_49,#Fg40_49Pct,#FgMade50Plus,#FgAtt50Plus,#Fg50PlusPct,#FgLng,#XpMade,#XpAtt,#XpPct,#FgAndXpPts
#Team Abbr.,,,,,,,,,,,,,,,,,,,,,
ARI,112.0,134.0,433.2,3.0,3.0,300.0,30.0,31.0,483.3,38.0,...,39.0,328.5,9.0,16.0,196.7,783.0,147.0,159.0,571.4,483.0
ATL,130.0,147.0,632.1,2.0,2.0,200.0,37.0,37.0,700.0,31.0,...,41.0,398.5,26.0,31.0,445.0,813.0,181.0,183.0,691.1,571.0
BAL,138.0,154.0,457.1,0.0,0.0,0.0,33.0,33.0,500.0,41.0,...,43.0,475.5,24.0,36.0,367.7,298.0,141.0,141.0,500.0,555.0
BUF,106.0,124.0,438.6,1.0,1.0,100.0,28.0,28.0,400.0,35.0,...,34.0,297.2,17.0,24.0,262.8,870.0,131.0,143.0,370.5,449.0
CAR,129.0,152.0,579.0,1.0,1.0,100.0,32.0,33.0,566.7,40.0,...,60.0,455.3,7.0,14.0,243.3,1006.0,167.0,176.0,678.0,554.0


In [155]:
puntTotals = teamTotals[["#Punts", "#PuntYds", "#PuntNetYds", "#PuntLng", "#PuntAvg", "#PuntNetAvg", 
                         "#PuntOOB", "#PuntDown", "#PuntIn20", "#PuntIn20Pct", "#PuntTB", 
                         "#PuntTBPct", "#PuntFC", "#PuntRet", "#PuntRetYds", "#PuntRetAvg"]]
puntTotals.head()

,#Punts,#PuntYds,#PuntNetYds,#PuntLng,#PuntAvg,#PuntNetAvg,#PuntOOB,#PuntDown,#PuntIn20,#PuntIn20Pct,#PuntTB,#PuntTBPct,#PuntFC,#PuntRet,#PuntRetYds,#PuntRetAvg
#Team Abbr.,,,,,,,,,,,,,,,,
ARI,309.0,13644.0,11957.0,1331.0,439.9,394.4,0.0,39.0,107.0,258.4,22.0,44.8,106.0,139.0,1687.0,94.4
ATL,230.0,10591.0,9425.0,1231.0,305.1,272.5,0.0,16.0,91.0,241.7,12.0,29.6,94.0,106.0,1166.0,70.4
BAL,241.0,11396.0,10332.0,336.0,240.2,208.3,0.0,38.0,101.0,170.0,17.0,29.2,58.0,118.0,1064.0,50.5
BUF,280.0,12649.0,11528.0,1131.0,225.6,209.1,0.0,38.0,82.0,164.3,19.0,42.8,67.0,146.0,1121.0,30.4
CAR,295.0,13417.0,11804.0,1483.0,402.1,356.8,0.0,43.0,103.0,286.7,24.0,108.2,87.0,136.0,1613.0,116.5


In [167]:
sptTurn = teamTotals[["#KB", "#KrFum", "#PrFum", "#FgBlk", "#XpBlk", "#PuntBlk"]]
sptTurn.head()

,#KB,#KrFum,#PrFum,#FgBlk,#XpBlk,#PuntBlk
#Team Abbr.,,,,,,
ARI,5.0,0.0,0.0,3.0,3.0,2.0
ATL,5.0,0.0,0.0,5.0,0.0,1.0
BAL,10.0,0.0,0.0,3.0,0.0,2.0
BUF,1.0,0.0,0.0,1.0,1.0,0.0
CAR,4.0,0.0,0.0,5.0,4.0,2.0


In [168]:
sptScores = teamTotals[["#KrTD", "#PrTD", "#FgMade", "#XpMade", "#FgAndXpPts", "#KoTD"]]
sptScores.head()

,#KrTD,#PrTD,#FgMade,#XpMade,#FgAndXpPts,#KoTD
#Team Abbr.,,,,,,
ARI,1.0,1.0,112.0,147.0,483.0,0.0
ATL,0.0,1.0,130.0,181.0,571.0,0.0
BAL,2.0,2.0,138.0,141.0,555.0,0.0
BUF,1.0,1.0,106.0,131.0,449.0,0.0
CAR,1.0,2.0,129.0,167.0,554.0,0.0


In [80]:
#Merge 8 CSV into 1 combined
# cumStats_combinedCSV= pd.concat([pd.read_csv])
# root = "C:.Users/Michael Diaz/Desktop/GitHub RBDS/Rutgers_Projects/RDSB_Sports/"
# path = os.path.join(root, "Cumulative Stats")
# # for path, csv, subdirs, files in os.walk(root):
# #     for name in files:
# #         print os.path.join(path, name)
# # cumStats = glob.glob("Cumulative Stats/*.csv")

# #Append all files to have dates

# # for file in cumStats:
# #     convert_DF = cumStats.insert(0, "Season")
# #     convert_DF.insert(2, "Season", len(rows))

# #     convert_DF.to_csv(cumStats.csv)
# #Loop to Combine ALl DF
# cumStats_DF = []
# for filename in sorted(cumStats):
#     cumStats_DF.append(pd.read_csv(filename))
# fullStats_DF = pd.concat(cumStats_DF, join="inner")

# fullStats_DF.to_csv("Cumulative_Stats_Combined.csv")
# fullStats_DF
# # print(cumStats)

,Unnamed: 0,Season,#Player ID,#LastName,#FirstName,#Jersey Num,#Position,#Height,#Weight,#Birth Date,...,#PuntDown,#PuntIn20,#PuntIn20Pct,#PuntTB,#PuntTBPct,#PuntFC,#PuntRet,#PuntRetYds,#PuntRetAvg,#GamesStarted
0,0,2014-2015 Regular,7350,Abdullah,Husain,39.0,DB,"6'0""",204.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,1,2014-2015 Regular,6899,Abdul-Quddus,Isa,42.0,DB,"6'1""",201.0,1989-08-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2,2014-2015 Regular,7861,Aboushi,Oday,75.0,G,"6'5""",315.0,1991-06-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,3,2014-2015 Regular,8062,Acho,Emmanuel,51.0,LB,"6'2""",240.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,4,2014-2015 Regular,6430,Acho,Sam,49.0,OLB,"6'3""",257.0,1988-09-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,5,2014-2015 Regular,8459,Acosta,Corey,25.0,K,"6'0""",190.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,2014-2015 Regular,6924,Adams,Davante,17.0,WR,"6'1""",215.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7,7,2014-2015 Regular,7039,Adams,Jeff,70.0,OT,"6'7""",305.0,1989-09-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
8,8,2014-2015 Regular,8134,Adams,Mike,76.0,OT,"6'7""",323.0,1990-03-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9,9,2014-2015 Regular,7170,Adams,Mike,29.0,SS,"5'11""",205.0,1981-03-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [ ]:
#Merge 8 CSV into 1 combined
# cumStats_combinedCSV= pd.concat([pd.read_csv])
# root = "C:.Users/Michael Diaz/Desktop/GitHub RBDS/Rutgers_Projects/RDSB_Sports/"
# path = os.path.join(root, "Cumulative Stats")
# for path, csv, subdirs, files in os.walk(root):
#     for name in files:
#         print os.path.join(path, name)
cumStats = glob.glob("Cumulative Stats/*.csv")

#Append all files to have dates

# for file in cumStats:
#     convert_DF = cumStats.insert(0, "Season")
#     convert_DF.insert(2, "Season", len(rows))

#     convert_DF.to_csv(cumStats.csv)
#Loop to Combine ALl DF
cumStats_DF = []
for filename in sorted(cumStats):
    cumStats_DF.append(pd.read_csv(filename))
fullStats_DF = pd.concat(cumStats_DF, join="inner")

fullStats_DF.to_csv("Cumulative_Stats_Combined.csv")
fullStats_DF
# print(cumStats)